In [464]:
import networkx as nx
import ndlib.models.ModelConfig as mc
import ndlib.models.epidemics as ep
from ndlib.viz.mpl.DiffusionTrend import DiffusionTrend
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [465]:


### CITE NETWORKX AND NDLIB !!!!!
def give_mean(data):
    return np.mean(list(data.values()))

def give_properties(G):
    # Model properties
    clustering_coefficient = give_mean(nx.clustering(G, nodes=None, weight=None))
    closeness_centrality = give_mean(nx.closeness_centrality(G, u=None, distance=None, wf_improved=True))
    betweenness_centrality = give_mean(nx.betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=None))
    degree_centrality = give_mean(nx.degree_centrality(G))
    # average_shortest_path_length = nx.average_shortest_path_length(G, weight=None, method=None)
    average_shortest_path_length = 0 

    properties = np.matrix([clustering_coefficient, closeness_centrality, betweenness_centrality, degree_centrality, average_shortest_path_length])

    return(properties)

def evaluate(beta, gamma, I, n_iterations, G, export_name="untitled.pdf", vis=False):
    # Model selection
    model = ep.SIRModel(G)

    # Model Configuration
    cfg = mc.Configuration()
    cfg.add_model_parameter('beta', beta)
    cfg.add_model_parameter('gamma', gamma)
    cfg.add_model_parameter("fraction_infected", I)

    model.set_initial_status(cfg)

    # Simulation execution
    iterations = model.iteration_bunch(n_iterations)
    trends = model.build_trends(iterations)

    # Visualization
    if vis == True:
        viz = DiffusionTrend(model, trends)
        viz.plot(export_name)

    return trends

In [466]:
# Generate Networks of equivalent Form
n = 100
p = 0.1
k = int(n * p)
m = int(n * p)

runs=10
properties_er, properties_ws, properties_ba = np.zeros((5,runs)), np.zeros((5,runs)), np.zeros((5,runs))
er_list, ws_list, ba_list = [], [], []

for i in range(0, runs):
    seed=int(i)
    er = nx.erdos_renyi_graph(n, p, seed)   
    properties_er[:,i] = give_properties(er)
    er_list.append(er)
    
print('done1')
for i in range(0, runs):
    seed=int(i)
    ws = nx.watts_strogatz_graph(n, k, p, seed)
    properties_ws[:,i] = give_properties(ws)
    ws_list.append(ws)

print('done2')
for i in range(0, runs):
    seed=int(i)
    ba = nx.barabasi_albert_graph(n, m, seed)
    properties_ba[:,i] = give_properties(ba)
    ba_list.append(ba)

done1
done2


In [467]:
properties_string = ["clustering_coefficient", "closeness_centrality", "betweenness_centrality", "degree_centrality", "average_shortest_path_length"]

plt.hist(properties_er[0,:])
plt.savefig('test.png')

In [468]:
# Simulate SIR spread on the network
plt.clf()
beta_list =  [1, 1/3, 1/6] 
gamma = 1/3
I0_list = [0.001, 0.01, 0.1]
n_iterations = 50

for I0 in I0_list:
    for beta in beta_list:
        er_X, er_Y, er_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations
        ws_X, ws_Y, ws_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations
        ba_X, ba_Y, ba_Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations

        for i in range(len(er_list)):
            er = er_list[i]
            ws = ws_list[i]
            ba = ba_list[i]

            trends_er = evaluate(beta, gamma, I0, n_iterations, er)
            trends_ws = evaluate(beta, gamma, I0, n_iterations, ws)
            trends_ba = evaluate(beta, gamma, I0, n_iterations, ba)

            er_X = np.add(er_X, trends_er[0]['trends']['node_count'][0])
            er_Y = np.add(er_Y, trends_er[0]['trends']['node_count'][1])
            er_Z = np.add(er_Z, trends_er[0]['trends']['node_count'][2])

            ws_X = np.add(ws_X, trends_ws[0]['trends']['node_count'][0]) 
            ws_Y = np.add(ws_Y, trends_ws[0]['trends']['node_count'][1]) 
            ws_Z = np.add(ws_Z, trends_ws[0]['trends']['node_count'][2]) 

            ba_X = np.add(ba_X, trends_ba[0]['trends']['node_count'][0]) 
            ba_Y = np.add(ba_Y, trends_ba[0]['trends']['node_count'][1]) 
            ba_Z = np.add(ba_Z, trends_ba[0]['trends']['node_count'][2]) 

        plt.ylim(0,1)
        plt.plot(er_Y/(runs*n))
        plot_string = 'plots2/er_beta-' + str(np.round(beta,2)) + '_gamma-' + str(np.round(gamma,2)) + '_I0-' + str(I0) + '.png'
        plt.savefig(plot_string)
        plt.clf()

        plt.ylim(0,1)
        plt.plot(ws_Y/(runs*n))
        plot_string = 'plots2/ws_beta-' + str(np.round(beta,2)) + '_gamma-' + str(np.round(gamma,2)) + '_I0-' + str(I0) + '.png'
        plt.savefig(plot_string)
        plt.clf()

        plt.ylim(0,1)
        plt.plot(ba_Y/(runs*n))
        plot_string = 'plots2/ba_beta-' + str(np.round(beta,2)) + '_gamma-' + str(np.round(gamma,2)) + '_I0-' + str(I0) + '.png'
        plt.savefig(plot_string)
        plt.clf()

/Users/jennadevries/anaconda3/lib/python3.10/site-packages/ndlib/models/DiffusionModel.py:169: UserWarning: The fraction_infected value is too low given the number of nodes of the selected graph: a single node will be set as infected
  warnings.warn(


In [469]:
# Dynamic vaccination campaign
# https://www.includehelp.com/python/load-csv-into-2d-matrix-with-numpy-for-plotting.aspx#:~:text=Loading%20CSV%20into%202D%20matrix,using%20the%20numpy%20array%20object.

def evaluate_vacc_step(beta, gamma, infected_nodes_initial, n_iterations, G, export_name="untitled.pdf", vis=False):
    infected_nodes = infected_nodes_initial
    model = ep.SIRModel(G) 
    for t in range(n_iterations):

        recovered_nodes = []
        
        # Model Configuration
        cfg = mc.Configuration()
        cfg.add_model_parameter('beta', beta)
        cfg.add_model_parameter('gamma', gamma)
        cfg.add_model_initial_configuration("Infected", infected_nodes)


        model.set_initial_status(cfg)

        # Simulation execution
        iterations = model.iteration_bunch(1)
        trends = model.build_trends(iterations)

        # Find direct neighbors of infected nodes
        neighbors_infected_nodes = []
        for node in infected_nodes:
            # print(node)
        
            for neighbor_node in G.neighbors(node):
                neighbors_infected_nodes.append(neighbor_node)

            # remove neighbor if neighbor already infected
            overlap = np.intersect1d(neighbors_infected_nodes, infected_nodes)
            for node in overlap:
                neighbors_infected_nodes.remove(node)
            
            # only have each neighbour once, even if it is shared by multiple nodes
            neighbors_infected_nodes = list(np.unique(neighbors_infected_nodes))

        number_of_infections = trends[0]['trends']['status_delta'][1][0]

        
        if number_of_infections > 0:
            """Random neighbors of the infected nodes are now infected, if the simulation asks for more infections than
            there are direct neighbors all direct neuighbors will be infected"""
            if len(neighbors_infected_nodes) > number_of_infections:
                added_infections = np.random.choice(neighbors_infected_nodes, number_of_infections, replace=False)
            else:
                added_infections = np.random.choice(neighbors_infected_nodes, len(neighbors_infected_nodes), replace=False)
            
            infected_nodes.extend(added_infections)
        
        if number_of_infections < 0:
            """Random infected nodes are now recovered, if the simulation asks for more recovering infected hosts than
            there are infected hosts all infected hosts will recover"""
            if len(infected_nodes) > np.abs(number_of_infections):
                removed_infections = np.random.choice(infected_nodes, np.abs(number_of_infections), replace=False)
            else:
                removed_infections = np.random.choice(infected_nodes, len(infected_nodes), replace=False)

            for removed_infection in removed_infections:
                infected_nodes.remove(removed_infection)
                G.remove_node(removed_infection)
                recovered_nodes.append(removed_infection)

    return G, infected_nodes, recovered_nodes
    

"Creating edgelist"
data = np.loadtxt(open("transmission_network.csv", "rb"), delimiter=";", skiprows=1)[:, 1:]
nonzero_x = np.nonzero(data)[0]
nonzero_y = np.nonzero(data)[1]

edges=[]
for i in range(len(nonzero_x)):
    edges.append((nonzero_x[i], nonzero_y[i]))

"Set parameters"
beta =  1
gamma = 1/3
n_iterations = 2
G = nx.from_edgelist(edges)
infected_nodes =  list(np.random.choice(list(G.nodes), 5, replace=False))

X, Y, Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations


for t in range(0,100):
    G_new, infected_nodes_new, recovered_nodes =evaluate_vacc_step(beta, gamma, infected_nodes, n_iterations, G)
    G=G_new
    infected_nodes=infected_nodes
    print(infected_nodes)

X = np.add(er_X, trends_er[0]['trends']['node_count'][0])
Y = np.add(er_Y, trends_er[0]['trends']['node_count'][1])
Z = np.add(er_Z, trends_er[0]['trends']['node_count'][2])


plt.ylim(0,1)
plt.plot(er_Y/(runs*n))
plt.savefig('vaccination.png')
plt.clf()

# SET INFECTED NODES
    # 


[181, 349, 112, 120, 322, 166, 174, 76, 368, 331, 103, 323, 24, 340, 119, 140, 353, 305, 38, 85, 344, 270]
[181, 349, 112, 120, 322, 166, 174, 76, 368, 331, 103, 323, 24, 340, 119, 140, 353, 305, 38, 85, 344, 270, 147, 277, 278, 105, 162, 89, 31, 97, 335, 345, 220, 79, 40, 114, 66, 62, 155, 21, 337, 146, 82, 287, 75, 276, 148, 168, 306, 195, 19, 69, 194, 127, 5, 80, 94, 9, 207, 133, 253, 136, 126, 302, 299, 295, 312, 233, 180, 326, 351, 307, 202, 294, 264, 27, 372, 359, 158, 78, 187, 251, 235, 261, 20, 154, 6, 95, 262, 102, 163, 111, 292, 88, 165, 167, 101, 81, 222, 169, 90, 84, 354, 230, 171, 130, 37, 248, 217, 289, 364, 13, 164, 170, 206, 279, 311, 123, 104, 93]
[181, 349, 112, 120, 322, 166, 174, 76, 368, 331, 103, 323, 24, 340, 119, 140, 353, 305, 38, 85, 344, 270, 147, 277, 278, 105, 162, 89, 31, 97, 335, 345, 220, 79, 40, 114, 66, 62, 155, 21, 337, 146, 82, 287, 75, 276, 148, 168, 306, 195, 19, 69, 194, 127, 5, 80, 94, 9, 207, 133, 253, 136, 126, 302, 299, 295, 312, 233, 180, 326

In [495]:
# Dynamic vaccination campaign
# https://www.includehelp.com/python/load-csv-into-2d-matrix-with-numpy-for-plotting.aspx#:~:text=Loading%20CSV%20into%202D%20matrix,using%20the%20numpy%20array%20object.

def evaluate_vacc_step(beta, gamma, infected_nodes_initial, n_iterations, G, tests_used=0, n_vacc=0, test_max=200):
    infected_nodes = infected_nodes_initial
    model = ep.SIRModel(G) 


    for _ in range(n_vacc):
        betweeness_centrality = nx.betweenness_centrality(G, k=None, normalized=True, weight=None, endpoints=False, seed=None)
        betweeness_centrality_values = list(betweeness_centrality.values())
        betweeness_centrality_ordered = sorted(list(betweeness_centrality.values()), reverse=True)

        target_node = infected_nodes[0]
        next_max = 0
        while target_node in infected_nodes:
            target_index = betweeness_centrality_values.index(betweeness_centrality_ordered[next_max])
            target_node = list(betweeness_centrality.keys())[target_index]
            tests_used += 1
            next_max += 1

        if tests_used < test_max:
            G.remove_node(target_node)


    for _ in range(n_iterations):

        recovered_nodes = []
        
        # Model Configuration
        cfg = mc.Configuration()
        cfg.add_model_parameter('beta', beta)
        cfg.add_model_parameter('gamma', gamma)
        cfg.add_model_initial_configuration("Infected", infected_nodes)


        model.set_initial_status(cfg)

        # Simulation execution
        iterations = model.iteration_bunch(1)
        trends = model.build_trends(iterations)

        # Find direct neighbors of infected nodes
        neighbors_infected_nodes = []
        for node in infected_nodes:
            # print(node)
        
            for neighbor_node in G.neighbors(node):
                neighbors_infected_nodes.append(neighbor_node)

            # remove neighbor if neighbor already infected
            overlap = np.intersect1d(neighbors_infected_nodes, infected_nodes)
            for node in overlap:
                neighbors_infected_nodes.remove(node)
            
            # only have each neighbour once, even if it is shared by multiple nodes
            neighbors_infected_nodes = list(np.unique(neighbors_infected_nodes))

        number_of_infections = trends[0]['trends']['status_delta'][1][0]

        
        if number_of_infections > 0:
            """Random neighbors of the infected nodes are now infected, if the simulation asks for more infections than
            there are direct neighbors all direct neuighbors will be infected"""
            if len(neighbors_infected_nodes) > number_of_infections:
                added_infections = np.random.choice(neighbors_infected_nodes, number_of_infections, replace=False)
            else:
                added_infections = np.random.choice(neighbors_infected_nodes, len(neighbors_infected_nodes), replace=False)
            
            infected_nodes.extend(added_infections)
        
        if number_of_infections < 0:
            """Random infected nodes are now recovered, if the simulation asks for more recovering infected hosts than
            there are infected hosts all infected hosts will recover"""
            if len(infected_nodes) > np.abs(number_of_infections):
                removed_infections = np.random.choice(infected_nodes, np.abs(number_of_infections), replace=False)
            else:
                removed_infections = np.random.choice(infected_nodes, len(infected_nodes), replace=False)

            for removed_infection in removed_infections:
                infected_nodes.remove(removed_infection)
                G.remove_node(removed_infection)
                recovered_nodes.append(removed_infection)

    return G, infected_nodes, recovered_nodes, tests_used
    

"Creating edgelist"
data = np.loadtxt(open("transmission_network.csv", "rb"), delimiter=";", skiprows=1)[:, 1:]
nonzero_x = np.nonzero(data)[0]
nonzero_y = np.nonzero(data)[1]

edges=[]
for i in range(len(nonzero_x)):
    edges.append((nonzero_x[i], nonzero_y[i]))

"Set parameters"
beta =  1
gamma = 1/3
n_iterations = 2
G = nx.from_edgelist(edges)
infected_nodes =  list(np.random.choice(list(G.nodes), 5, replace=False))
tests_used = 0
n_vacc = 10
test_max = 200

X, Y, Z = [0]*n_iterations, [0]*n_iterations, [0]*n_iterations


for t in range(0,100):
    G_new, infected_nodes_new, recovered_nodes, tests_used_new = evaluate_vacc_step(beta, gamma, infected_nodes, n_iterations, G, tests_used, n_vacc, test_max)
    G=G_new
    infected_nodes=infected_nodes
    tests_used = tests_used_new
    print(t)

X = np.add(er_X, trends_er[0]['trends']['node_count'][0])
Y = np.add(er_Y, trends_er[0]['trends']['node_count'][1])
Z = np.add(er_Z, trends_er[0]['trends']['node_count'][2])


plt.ylim(0,1)
plt.plot(er_Y/(runs*n))
plt.savefig('vaccination.png')
plt.clf()




TypeError: 'int' object is not subscriptable